#### Read

#### Write

#### CSV

In [20]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
val myManualSchema = new StructType(Array(
new StructField("DEST_COUNTRY_NAME", StringType, true),
new StructField("ORIGIN_COUNTRY_NAME", StringType, true),
new StructField("count", LongType, false)
))

myManualSchema = StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,false))


StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,false))

In [21]:
val csvFile=spark.read.format("csv")
.option("header", "true")
.option("mode", "FAILFAST")
.schema(myManualSchema)
.load("/home/koushik/git/spark/input/Spark-The-Definitive-Guide/flight-data/csv/2010-summary.csv")

csvFile = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [7]:
csvFile.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
+-----------------+-------------------+-----+
only showing top 2 rows



In [5]:
//write
csvFile.write.format("csv").mode("overwrite").option("sep", "\t").save("/home/koushik/git/spark/output/Spark-The-Definitive-Guide/flight-data/csv/2010-summary.tsv")

#### JSON

In [11]:
val jsonFile=spark.read
.format("json")
.option("mode", "FAILFAST")
.option("multiLine", false)
.schema(myManualSchema)
.load("/home/koushik/git/spark/input/Spark-The-Definitive-Guide/flight-data/json/2010-summary.json")

jsonFile = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [8]:
jsonFile.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
+-----------------+-------------------+-----+
only showing top 2 rows



In [13]:
csvFile.write.format("json").mode("overwrite")
.save("/home/koushik/git/spark/output/Spark-The-Definitive-Guide/flight-data/json/2010-summary.json")

#### Parquet

In [15]:
val parquetFile=spark.read.format("parquet")
.load("/home/koushik/git/spark/input/Spark-The-Definitive-Guide/flight-data/parquet/2010-summary.parquet")

parquetFile = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


lastException: Throwable = null


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [16]:
parquetFile.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
+-----------------+-------------------+-----+
only showing top 2 rows



In [17]:
csvFile.write.format("parquet").mode("overwrite")
.save("/home/koushik/git/spark/output/Spark-The-Definitive-Guide/flight-data/parquet/2010-summary.parquet")

#### Orc

In [2]:
val orcFile=spark.read.format("orc").load("/home/koushik/git/spark/input/Spark-The-Definitive-Guide/flight-data/orc/2010-summary.orc")

orcFile = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [1]:
orcFile.show(2)

Name: Compile Error
Message: <console>:26: error: not found: value orcFile
       orcFile.show(2)
       ^

StackTrace: 

In [21]:
csvFile.write.format("orc").mode("overwrite").save("/home/koushik/git/spark/output/Spark-The-Definitive-Guide/flight-data/orc/2010-summary.orc")

#### SQL Databases

In [7]:
val dbDataFrame = spark.read.format("jdbc")
    .option("url", "jdbc:mysql://127.0.0.1:3306?useSSL=false")
    .option("dbtable","employees.employees")
    .option("user", "root")
    .option("driver", "com.mysql.jdbc.Driver")
    //.option("password", "root")
    .load()

dbDataFrame = [emp_no: int, birth_date: date ... 4 more fields]


[emp_no: int, birth_date: date ... 4 more fields]

In [9]:
dbDataFrame.select("*").distinct().show(5)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10040|1959-09-13|     Weiyi|  Meriste|     F|1993-02-14|
| 10268|1958-06-03|    Nishit|   Siochi|     M|1986-12-17|
| 10288|1959-06-02|    Selwyn|    Perri|     M|1994-08-29|
| 10463|1954-05-20|       Ung|   Zaiane|     M|1987-12-25|
| 10677|1963-02-20| Alejandra|    Perng|     F|1996-10-07|
+------+----------+----------+---------+------+----------+
only showing top 5 rows



In [10]:
//Query Pushdown . Gets only the resultant data from jdbc
dbDataFrame.filter("gender ='M'").explain

== Physical Plan ==
*(1) Scan JDBCRelation(employees.employees) [numPartitions=1] [emp_no#23,birth_date#24,first_name#25,last_name#26,gender#27,hire_date#28] PushedFilters: [*IsNotNull(gender), *EqualTo(gender,M)], ReadSchema: struct<emp_no:int,birth_date:date,first_name:string,last_name:string,gender:string,hire_date:date>


In [11]:
//Passing the whole query
val pushdownQuery = """(select * from employees where gender='F' AND birth_date > '1965-01-01' and hire_date < '1986-01-01')AS flight_info"""
val dbDataFrame = spark.read.format("jdbc")
    .option("url", "jdbc:mysql://127.0.0.1:3306?useSSL=false")
    .option("dbtable","employees.employees")
    .option("user", "root")
    .option("driver", "com.mysql.jdbc.Driver")
    .load()

pushdownQuery = (select * from employees where gender='F' AND birth_date > '1965-01-01' and hire_date < '1986-01-01')AS flight_info
dbDataFrame = [emp_no: int, birth_date: date ... 4 more fields]


[emp_no: int, birth_date: date ... 4 more fields]

In [13]:
dbDataFrame.show(2)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10001|1953-09-02|    Georgi|  Facello|     M|1986-06-26|
| 10002|1964-06-02|   Bezalel|   Simmel|     F|1985-11-21|
+------+----------+----------+---------+------+----------+
only showing top 2 rows



In [15]:
val dbDataFramePart = spark.read.format("jdbc")
    .option("driver", "com.mysql.jdbc.Driver")
    .option("url", "jdbc:mysql://127.0.0.1:3306?useSSL=false")
    .option("dbtable","employees.employees")
    .option("user", "root")    
    .option("numPartitions", 10)// Reading from databases in parallel
    .load()

dbDataFramePart = [emp_no: int, birth_date: date ... 4 more fields]


[emp_no: int, birth_date: date ... 4 more fields]

In [7]:
val props = new java.util.Properties
props.setProperty("driver", "com.mysql.jdbc.Driver")
//props.setProperty("url", "jdbc:mysql://127.0.0.1:3306?useSSL=false")
//props.setProperty("dbtable","employees.dept_emp")
props.setProperty("user", "root")
val predicates = Array(
"hire_date < '1986-01-01' OR birth_date > '1965-01-01'",
"hire_date < '1980-01-01' OR birth_date > '1960-01-01'")

props = {user=root, driver=com.mysql.jdbc.Driver}
predicates = Array(hire_date < '1986-01-01' OR birth_date > '1965-01-01', hire_date < '1980-01-01' OR birth_date > '1960-01-01')


Array(hire_date < '1986-01-01' OR birth_date > '1965-01-01', hire_date < '1980-01-01' OR birth_date > '1960-01-01')

In [26]:
spark.read.jdbc("jdbc:mysql://127.0.0.1:3306?useSSL=false","employees.dept_emp", props).show(2)
spark.read.jdbc("jdbc:mysql://127.0.0.1:3306?useSSL=false","employees.dept_emp", props).rdd.getNumPartitions

+------+-------+----------+----------+
|emp_no|dept_no| from_date|   to_date|
+------+-------+----------+----------+
| 10001|   d005|1986-06-26|9999-01-01|
| 10002|   d007|1996-08-03|9999-01-01|
+------+-------+----------+----------+
only showing top 2 rows



1

In [9]:
val colName = "emp_no"
val lowerBound = 0L
val upperBound = 999999L // this is the max count in our database
val numPartitions = 10

colName = emp_no
lowerBound = 0
upperBound = 999999
numPartitions = 10


10

In [25]:
spark.read.jdbc("jdbc:mysql://127.0.0.1:3306?useSSL=false","employees.dept_emp",colName,lowerBound,upperBound,
                numPartitions,props).count()
spark.read.jdbc("jdbc:mysql://127.0.0.1:3306?useSSL=false","employees.dept_emp",colName,lowerBound,upperBound,
                numPartitions,props).rdd.getNumPartitions

10

#### Text

In [17]:
spark.read.textFile("/home/koushik/git/spark/input/Spark-The-Definitive-Guide/flight-data/csv/2010-summary.csv")
.selectExpr("split(value, ',') as rows").show(2)

+--------------------+
|                rows|
+--------------------+
|[DEST_COUNTRY_NAM...|
|[United States, R...|
+--------------------+
only showing top 2 rows



In [22]:
csvFile.select("DEST_COUNTRY_NAME").
write.text("/home/koushik/git/spark/output/Spark-The-Definitive-Guide/flight-data/csv/2010-summary.txt")

#### Partitioning

In [23]:
csvFile.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME")
.save("/home/koushik/git/spark/output/Spark-The-Definitive-Guide/flight-data/partitionedcsv/2010-summary.parquet")

#### Bucketing

In [24]:
csvFile.write.format("parquet").mode("overwrite")
.bucketBy(10, "count").saveAsTable("bucketedFiles")